In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
import random
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, PowerTransformer,QuantileTransformer ,LabelEncoder, OneHotEncoder, MinMaxScaler
import seaborn as sns
! pip install prince
import prince

! pip install optuna
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [3]:
path = '/content/drive/Shareddrives/dacon/Playground/'
train = pd.read_csv(path + 'train.csv', index_col=['id'])
test = pd.read_csv(path + 'test.csv', index_col=['id'])
submission = pd.read_csv(path + 'sample_submission.csv', index_col=['id'])

train_data = train.iloc[:,:-1]
train_target = train.iloc[:, -1]
test_data = test.copy()

In [4]:
cat_features = [feature for feature in train_data.columns if 'cat' in feature]
cont_features = [feature for feature in train_data.columns if 'cont' in feature]
for feature in cat_features:
    train_data[feature] = train_data[feature].astype('category')
    test_data[feature] = test_data[feature].astype('category')

In [5]:
a = pd.get_dummies(train_data[cat_features])
b = pd.get_dummies(test_data[cat_features])

missing_cols = set(a.columns) - set(b.columns)
for c in missing_cols:
  b[c] = 0

b = b[a.columns]

In [6]:
trainset = pd.concat([train_data.loc[:, 'cont0':], a], axis = 1)
testset = pd.concat([test_data.loc[:, 'cont0':], b], axis = 1)

In [7]:
testset

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cat0_A,cat0_B,cat1_A,cat1_B,cat2_A,cat2_B,cat3_A,cat3_B,cat3_C,cat3_D,cat4_A,cat4_B,cat4_C,cat4_D,cat5_A,cat5_B,cat5_C,cat5_D,cat6_A,cat6_B,cat6_C,cat6_D,cat6_E,cat6_G,cat6_H,cat6_I,cat7_A,cat7_B,cat7_C,cat7_D,cat7_E,cat7_F,cat7_G,cat7_I,cat8_A,cat8_B,cat8_C,cat8_D,cat8_E,cat8_F,cat8_G,cat9_A,cat9_B,cat9_C,cat9_D,cat9_E,cat9_F,cat9_G,cat9_H,cat9_I,cat9_J,cat9_K,cat9_L,cat9_M,cat9_N,cat9_O
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.331261,0.551131,0.486284,0.156058,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
5,0.402404,0.617546,0.296852,0.187440,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
15,0.530677,0.618126,0.760865,0.199872,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
16,0.439653,0.487403,0.715786,0.192808,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
17,0.389844,0.752760,0.315269,0.191734,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499987,0.919265,0.111147,0.199583,0.181354,0.277365,0.963678,0.240482,0.686462,0.915165,0.848878,0.459598,0.590327,0.864873,0.425258,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
499990,0.362875,-0.020197,0.469025,0.336185,0.523174,0.232072,0.363421,0.694092,0.137002,0.319465,0.364527,0.388908,0.664357,0.224215,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
499991,0.463060,0.740421,0.446293,0.411387,0.517103,0.432927,0.811876,0.328398,0.496017,0.538779,0.466338,0.643869,0.749590,0.457702,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [12]:
def create_stratified_folds_for_regression(data_df, data_target, n_splits=4):
    """
    @param data_df: training data to split in Stratified K Folds for a continous target value
    @param n_splits: number of splits
    @return: the training data with a column with kfold id
    """
    data_df = pd.concat([data_df, data_target], axis = 1)
    data_df['kfold'] = -1
    # randomize the data
    data_df = data_df.sample(frac=1, random_state=42).reset_index(drop=True)
    # calculate the optimal number of bins based on log2(data_df.shape[0])
    num_bins = np.int(np.floor(1 + np.log2(len(data_df))))
    # bins value will be the equivalent of class value of target feature used by StratifiedKFold to 
    # distribute evenly the classed over each fold
    data_df.loc[:, "bins"] = pd.cut(pd.to_numeric(data_df['target'], downcast="signed"), bins=num_bins, labels=False)
    kf = StratifiedKFold(n_splits=n_splits, random_state=42)
    
    # set the fold id as a new column in the train data
    for f, (t_, v_) in enumerate(kf.split(X=data_df, y=data_df.bins.values)):
        data_df.loc[v_, 'kfold'] = f
    
    # drop the bins column (no longer needed)
    data_df = data_df.drop("bins", axis=1)
    
    return data_df

In [32]:
def objectiveXGB(trial: Trial, X):
    # param = {
    #     "n_estimators" : 500,
    #     'max_depth':trial.suggest_int('max_depth', 8, 16),
    #     'min_child_weight':trial.suggest_int('min_child_weight', 1, 300),
    #     'gamma':trial.suggest_int('gamma', 1, 3),
    #     'learning_rate': 0.01,
    #     'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.05),
    #     'nthread' : -1,
    #     'tree_method': 'gpu_hist',
    #     'predictor': 'gpu_predictor',
    #     'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
    #     'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
    #     'subsample': trial.suggest_discrete_uniform('subsample', 0.5,1, 0.05 ),
    #     'random_state': 42
    # }

    xgb_params = {}
    xgb_params['objective'] = 'reg:squarederror'
    xgb_params['n_estimators'] = 250
    xgb_params['max_depth'] = trial.suggest_int('max_depth', 3, 16)
    xgb_params['min_child_weight'] = trial.suggest_int('min_child_weight', 1, 300)
    xgb_params['gamma'] = trial.suggest_discrete_uniform('gamma', 0.5, 10, 0.5)
    xgb_params['learning_rate'] = 0.01
    xgb_params['colsample_bytree'] = trial.suggest_discrete_uniform('colsample_bytree',0.5, 1, 0.05)
    xgb_params['tree_method'] = 'gpu_hist'
    xgb_params['gpu_id'] = 0
    xgb_params['lambda'] = trial.suggest_loguniform('lambda', 1e-3, 10.0)
    xgb_params['alpha'] = trial.suggest_loguniform('alpha', 1e-3, 10.0)
    xgb_params['subsample'] = trial.suggest_discrete_uniform('subsample', 0.5,1, 0.05 )
    xgb_params['random_state'] = 42

    model = XGBRegressor(**xgb_params)

    score = cv_evaluation_regression(X, model)

    return np.sqrt(score)

In [33]:
def cv_evaluation_regression(df, model):
    n_fold = 4
    oof = np.zeros((len(df),))
    for fold in tqdm(range(n_fold)):
        val_ind = df[df.kfold == fold].index
        train = df[df.kfold != fold].reset_index(drop=True)
        valid = df[df.kfold == fold].reset_index(drop=True)

        features = [feature for feature in df.columns if feature not in ['id', 'target', 'kfold']]

        X_train = train[features]
        y_train = train['target']
        X_valid = valid[features]
        y_valid = valid['target']

        model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 500, verbose=False)
        preds = model.predict(valid[features])
        mse = mean_squared_error(y_valid, preds)
        
        oof[val_ind] = preds

    mse_oof = mean_squared_error(oof, df['target'])
    return np.sqrt(mse_oof)

In [34]:
train_cv = create_stratified_folds_for_regression(trainset, train_target)

In [35]:
study = optuna.create_study(direction = 'minimize')
study.optimize(lambda trial : objectiveXGB(trial, train_cv), n_trials=200)

[I 2021-02-12 12:08:43,268] A new study created in memory with name: no-name-658422bd-6c81-494c-8a72-13865a59967e


[I 2021-02-12 12:09:02,887] Trial 0 finished with value: 1.0133884386321206 and parameters: {'max_depth': 6, 'min_child_weight': 49, 'gamma': 10.0, 'colsample_bytree': 0.65, 'lambda': 0.0010949651136146162, 'alpha': 0.09977232557933094, 'subsample': 0.8500000000000001}. Best is trial 0 with value: 1.0133884386321206.


[I 2021-02-12 12:09:18,276] Trial 1 finished with value: 1.015214464056008 and parameters: {'max_depth': 4, 'min_child_weight': 290, 'gamma': 2.0, 'colsample_bytree': 0.55, 'lambda': 0.021063895754423224, 'alpha': 0.33131284430175034, 'subsample': 0.75}. Best is trial 0 with value: 1.0133884386321206.


[I 2021-02-12 12:09:57,650] Trial 2 finished with value: 1.011740040622605 and parameters: {'max_depth': 11, 'min_child_weight': 33, 'gamma': 4.0, 'colsample_bytree': 0.55, 'lambda': 0.009921665616035328, 'alpha': 0.03729748652007931, 'subsample': 0.8}. Best is trial 2 with value: 1.011740040622605.


[I 2021-02-12 12:11:21,055] Trial 3 finished with value: 1.012364554601031 and parameters: {'max_depth': 15, 'min_child_weight': 18, 'gamma': 9.5, 'colsample_bytree': 0.55, 'lambda': 2.5222234288321475, 'alpha': 0.005428533097775699, 'subsample': 0.8500000000000001}. Best is trial 2 with value: 1.011740040622605.


...

[I 2021-02-12 14:31:00,068] Trial 195 finished with value: 1.011096051107657 and parameters: {'max_depth': 16, 'min_child_weight': 258, 'gamma': 5.5, 'colsample_bytree': 0.8500000000000001, 'lambda': 0.02778853436666757, 'alpha': 3.885180348825727, 'subsample': 0.95}. Best is trial 120 with value: 1.0110843817343718.


[I 2021-02-12 14:31:45,805] Trial 196 finished with value: 1.0110918006205747 and parameters: {'max_depth': 16, 'min_child_weight': 265, 'gamma': 5.0, 'colsample_bytree': 0.8500000000000001, 'lambda': 0.018048655850317116, 'alpha': 3.0562453346231644, 'subsample': 0.95}. Best is trial 120 with value: 1.0110843817343718.


[I 2021-02-12 14:32:31,815] Trial 197 finished with value: 1.0110995217748582 and parameters: {'max_depth': 16, 'min_child_weight': 260, 'gamma': 5.0, 'colsample_bytree': 0.8500000000000001, 'lambda': 0.01840134144102593, 'alpha': 3.25306189197591, 'subsample': 0.95}. Best is trial 120 with value: 1.0110843817343718.


[I 2021-02-12 14:33:18,362] Trial 198 finished with value: 1.0111426560787302 and parameters: {'max_depth': 16, 'min_child_weight': 266, 'gamma': 5.0, 'colsample_bytree': 0.8500000000000001, 'lambda': 0.03821871318204003, 'alpha': 2.7301103912272877, 'subsample': 1.0}. Best is trial 120 with value: 1.0110843817343718.


[I 2021-02-12 14:34:04,506] Trial 199 finished with value: 1.0110976203807962 and parameters: {'max_depth': 16, 'min_child_weight': 256, 'gamma': 5.0, 'colsample_bytree': 0.8500000000000001, 'lambda': 0.027918174050962127, 'alpha': 4.370984168146186, 'subsample': 0.95}. Best is trial 120 with value: 1.0110843817343718.


In [36]:
optuna.visualization.plot_optimization_history(study)

In [37]:
study.best_params

{'alpha': 2.6457221052775166,
 'colsample_bytree': 0.8500000000000001,
 'gamma': 4.5,
 'lambda': 0.35098294093252796,
 'max_depth': 16,
 'min_child_weight': 269,
 'subsample': 0.9}

In [ ]:
study.optimize(lambda trial : objectiveXGB(trial, trainset, train_target), n_trials=50)

[18:03:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-02-11 18:04:41,344] Trial 124 finished with value: 0.7973725633743134 and parameters: {'max_depth': 16, 'min_child_weight': 126, 'gamma': 3, 'colsample_bytree': 0.7, 'lambda': 0.0010020890851938206, 'alpha': 0.654165429229869, 'subsample': 0.95}. Best is trial 106 with value: 0.6489867398537718.


[18:04:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-02-11 18:06:28,285] Trial 125 finished with value: 0.6984567989916053 and parameters: {'max_depth': 16, 'min_child_weight': 13, 'gamma': 3, 'colsample_bytree': 0.7, 'lambda': 0.0014598412531115997, 'alpha': 1.0290549422024986, 'subsample': 0.95}. Best is trial 106 with value: 0.6489867398537718.


[18:06:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

[18:46:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-02-11 18:48:45,520] Trial 143 finished with value: 0.6614859486506139 and parameters: {'max_depth': 16, 'min_child_weight': 4, 'gamma': 3, 'colsample_bytree': 0.7, 'lambda': 0.001002901185106622, 'alpha': 1.3593875019657307, 'subsample': 0.95}. Best is trial 106 with value: 0.6489867398537718.


[18:48:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[I 2021-02-11 18:52:21,192] Trial 144 finished with value: 0.6489867398537718 and parameters: {'max_depth': 16, 'min_child_weight': 1, 'gamma': 3, 'colsample_bytree': 0.7, 'lambda': 0.0013980366230734998, 'alpha': 1.6074817118503373, 'subsample': 0.95}. Best is trial 106 with value: 0.6489867398537718.


[18:52:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
